In [ ]:
# Niveen Abdul-Mohsen (bvn9ad)
# Reinforcement Learning (CS 4771) - Figure 3.5 and Figure 4.1
# Value Iteration and Policy Iteration Implementation
# i used numpy for numerical operations and matplotlib for plotting